<a href="https://colab.research.google.com/github/tensorflow-project/FineTuning/blob/main/two_concepts/training_two_concepts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/tensorflow-project/FineTuning

Cloning into 'FineTuning'...
remote: Enumerating objects: 1326, done.
remote: Counting objects: 100% (54/54), done.
remote: Compressing objects: 100% (54/54), done.
remote: Total 1326 (delta 31), reused 0 (delta 0), pack-reused 1272
Receiving objects: 100% (1326/1326), 155.04 MiB | 23.27 MiB/s, done.
Resolving deltas: 100% (979/979), done.


In [ ]:
import sys
import os
import numpy as np
from google.colab import drive

### agree to mounting your Google Drive
drive.mount("/content/drive")

py_file_location = "/content/FineTuning/two_concepts"
sys.path.append(os.path.abspath(py_file_location))
py_file_location = "/content/FineTuning/models"
sys.path.append(os.path.abspath(py_file_location))

import textual_inversion_two_concepts as txt

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
By using this model checkpoint, you acknowledge that its usage is subject to the terms of the CreativeML Open RAIL-M license at https://raw.githubusercontent.com/CompVis/stable-diffusion/main/LICENSE
13911/13911 [==============================] - 0s 0us/step


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


136824240/136824240 [==============================] - 1s 0us/step


In [ ]:
### EXECUTE THE FOLLOWING TWO BLOCKS ONLY IF YOU WANT TO CONTINUE TRAINING WITH SAVED WEIGTHS
### choose where to load the weights from, either from your google drive or you load our pretrained weights
### make sure to insert the exact name of your weight.npy
path = '/content/drive/MyDrive/1weights_with_two_concepts.npy'

In [ ]:
###load the array of the weights of the text encoder from the training
text_encoder_weights = np.load(path, allow_pickle=True)

### Set the weights of the text encoder
txt.stable_diffusion.text_encoder.set_weights(text_encoder_weights)

In [ ]:
### training
txt.trainer.fit(
    txt.train_ds,
    epochs=1,
)

220/220 [==============================] - 432s 2s/step - loss: 0.0553


In [ ]:
### choose where to save your newly generated weights
path = '/content/drive/MyDrive/1weights_with_two_concepts.npy'

Mounted at /content/drive


In [ ]:
###get the weights of the text encoder and save the to Google Drive
text_encoder_weights = np.array(txt.stable_diffusion.text_encoder.get_weights(), dtype=object)

### Save the weights array to a file on your Google Drive
np.save(path, text_encoder_weights)

<ipython-input-5-0443ae8bd6c9>:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  text_encoder_weights = np.array(txt.stable_diffusion.text_encoder.get_weights())
